In [1]:
import json
import tqdm
import hashlib
from typing import (
    Union,
    Sequence
)

# function hub
def read_json_file(file_path): 
    ''' opens a json file, return its content save in a dictionary
    @input: file path to the json file
    @output: a dictionary of json file content
    '''
    file = open(file_path, 'r')
    values = json.load(file)
    file.close()
    return values

def hash_elems(*a):
    
    
    h = hashlib.sha256()
    h.update("|".encode("utf-8"))
    
    for x in a:

        if not x:
            # This case captures empty lists and None, nicely guaranteeing that we don't
            # need to do a recursive call if the list is empty. So we need a string to
            # feed in for both of these cases. "None" would be a Python-specific thing,
            # so we'll go with the more JSON-ish "null".
            hash_me = "null"

        elif isinstance(x, str):
            # strings are iterable, so it's important to handle them before the following check
            hash_me = x
        elif isinstance(x, Sequence):
             # The simplest way to deal with lists, tuples, and such are to crunch them recursively.
            hash_me = str(hash_elems(*x))           
        else:
            hash_me = str(x)
        h.update((hash_me + "|").encode("utf-8"))

    # Note: the returned value will range from [1,Q), because zeros are bad
    # for some of the nonces. (g^0 == 1, which would be an unhelpful thing
    # to multiply something with, if you were trying to encrypt it.)

    # Also, we don't need the checked version of int_to_q, because the
    # modulo operation here guarantees that we're in bounds.
    # return int_to_q_unchecked(
    #     1 + (int.from_bytes(h.digest(), byteorder="big") % Q_MINUS_ONE)
    # )

    return 1 + (int.from_bytes(h.digest(), byteorder="big") % (q - 1))

    

In [2]:

# file path configuration
constants_file_path = 'results/constants.json'
context_file_path = 'results/context.json'
ballot_file_path = 'results/encrypted_ballots/ballot_ballot-af0a2a64-c786-11ea-a311-acde48001122.json'


# basic parameters
g = 0
p = 0
q = 0
r = 0
gInv = 0
K = 0

In [3]:


constants = read_json_file(constants_file_path)
#print(constants)

g = constants['g']
p = constants['p']
q = constants['q']
q_test = pow(2, 256) - 189

r = constants['r']
gInv = constants['gInv']

#print('g:'+str(g))
#print('p:'+str(p))
#print('q:'+str(q))

context = read_json_file(context_file_path)
#print(context)

K = int(context['elgamalPublicKey'])
QBar = context['cryptoExtendedBaseHash']

print(K)


8902228262352507482376024745531485971443340210464286875179634320227592743274706098381064532883160461211626951858008519076836811880985097802420111949675931804135492341936214204781882140780957252442215380957624854841507680832366704118530338114603192117215467177814401302903243141053544520818713824008399256926707721375104566953365657924590153186853581901380188498253951405612841921342606069743021006123304700009120712081439516853013896197939798539033738691446135507942972748654043241628612519782688488132553511658831880620895588870861811987109148280056555228733158308943437556966475596801135473289552751666989626057427535108529692297275166884973597872260136554561439983154503891840348240685210820490824716553029531446855846702152825665048487339721674399980978668881945855597259493113647141520484186616308305661333443119826505716914118192019463320022197012998539863235037320622230878221413775934537552183408339688696853598918360032997804775400094302732993271962282845522421257766586314548117938780349454

In [4]:
# one sample data
ballot = read_json_file(ballot_file_path)

error = False

contests = ballot['contests']
for contest in contests:
    
    ballotSelections = contest['ballotSelections']
    for ballotSelection in ballotSelections:


        objectId = ballotSelection['objectId']

        alpha = int(ballotSelection['message']['alpha'])
        beta = int(ballotSelection['message']['beta'])

        a0 = int(ballotSelection['proof']['a0'])
        a1 = int(ballotSelection['proof']['a1'])
        b0 = int(ballotSelection['proof']['b0'])
        b1 = int(ballotSelection['proof']['b1'])
        c0 = int(ballotSelection['proof']['c0'])
        c1 = int(ballotSelection['proof']['c1'])
        v0 = int(ballotSelection['proof']['v0'])
        v1 = int(ballotSelection['proof']['v1'])

        alphaPow = pow(alpha, q, p)
        betaPow = pow(beta, q, p)
        a0Pow = pow(a0, q, p)
        b0Pow = pow(b0, q, p)
        a1Pow = pow(a1, q, p)
        b1Pow = pow(b1, q, p)

        #print("a0:"+str(a0))
        #print("alpha_1:"+str(alpha_1))
        # 3.1 The given values alpha, beta, a0, b0, a1, and b1 are all in the set Zpr.
        if alpha < 0 or alpha >= p:
            print("alpha is out of range.")
            error = True  
        if beta < 0 or beta >= p:
            print("beta is out of range.")
            error = True
        if a0 < 0 or a0 >= p:
            print("a0 is out of range.")
            error = True
        if a1 < 0 or a1 >= p:
            print("a1 is out of range.")
            error = True
        if b0 < 0 or b0 >= p:
            print("b0 is out of range.")
            error = True
        if b1 < 0 or b1 >= p:
            print("b1 is out of range.")
            error = True           
        if alphaPow != 1:
            print("alpha^q mod p does not equal to 1.")
            error = True
        if betaPow != 1:
            print("beta^q mod p does not equal to 1.")
            error = True
        if a0Pow != 1:
            print("a0^q mod p does not equal to 1.")
            error = True
        if a1Pow != 1:
            print("a1^q mod p does not equal to 1.")
            error = True
        if b0Pow != 1:
            print("b0^q mod p does not equal to 1.")
            error = True
        if b1Pow != 1:
            print("b1^q mod p does not equal to 1.")
            error = True   

        # 3.2 The challenge c is computed as c = H(Q, (alpha, beta), (a0, b0), (a1, b1))
        # tempString = str(QBar) + str(alpha) + str(beta) + str(a0) + str(b0) + str(a1) + str(b1)
        # tempList = (QBar, (alpha, beta), (a0, b0), (a1, b1))
        c = hash_elems(alpha, beta, a0, b0, a1, b1)
        #print(c)
        
        #print((c0 + c1) % q)
        
        if c != (c0 + c1) % q:
            print("c != (c0 + c1) % q.")
            error = True

        # 3.3 The given values c0, c1, v0, and v1 are each in the set Zq.
        if c0 < 0 or c0 >= q:
            print("c0 is out of range.")
            error = True
        if c1 < 0 or c1 >= q:
            print("c1 is out of range.")
            error = True
        if v0 < 0 or v0 >= q:
            print("v0 is out of range.")
            error = True
        if v1 < 0 or v1 >= q:
            print("v1 is out of range.")
            error = True       

        # 1st equation: gv0=a0alphac0 mod p
        g_v0_ls = pow(g, v0, p) % p
        g_v0_rs = (a0 * pow(alpha, c0, p)) % p
    
        # 2nd equation: gv1=a1alphac1 mod p
        g_v1_ls = pow(g, v1, p) % p
        g_v1_rs = (a1 * pow(alpha, c1, p)) % p

        # 3rd equation: Kv0=b0betac0 mod p
        K_v0_ls = pow(K, v0, p) % p
        K_v0_rs = (b0 * pow(beta, c0, p)) % p
    
        # 4th equation: gc1Kv1=b1betac1 mod p
        fourth_ls = (pow(g, c1, p) * pow(K, v1, p) % p)
        fourth_rs = (b1 * pow(beta, c1, p)) % p

        # checking
        if g_v0_ls != g_v0_rs:
            print("g_vo != a_0*alpha^c_0 at cast_ballots {i}, contest {j}, selection {k}")
            error = True
        if g_v1_ls != g_v1_rs:
            print("g_v1 != a_1*alpha^c_1")
            error = True
        if K_v0_ls != K_v0_rs:
            print("K_v0 != b_0*alpha^c_0")
            error = True
        if fourth_ls != fourth_rs:
            print("g_c1 * K_v1 != (b1 * beta^c1) % p")
            error = True

                                               
if not error:
    print("success")   



success


In [ ]:

# Green box 4

contests = ballot['contests']
for contest in contests:
    
    # total parameters
    big_a = int(contest['proof']['a'])
    big_b = int(contest['proof']['b'])
    big_c = int(contest['proof']['c'])
    big_v = int(contest['proof']['v'])
    big_constant = int(contest['proof']['constant'])
    big_alpha = 1
    big_beta = 1
  
    ballotSelections = contest['ballotSelections']
    for ballotSelection in ballotSelections:
        
        alpha = int(ballotSelection['message']['alpha'])
        beta = int(ballotSelection['message']['beta'])  
        big_alpha = big_alpha * alpha % p
        big_beta = big_beta * beta % p
    
    temp_c = hash_elems(big_alpha, big_beta, big_a, big_b)
    print("---------------------------------------------------------------")
    #print(temp_c)
    #print(big_c)
    
    
    g_v_ls = pow(g, big_v) % p
    a_ac_rs = (big_a * pow(big_alpha, big_c)) % p 
    
    print(g_v_ls)
    print(a_ac_rs)
    
    #g_l_k_ls = (pow(g, big_c) % p) * (pow(K, big_v) % p)
    #b_bc_rs = (big_b * pow(big_beta, big_c)) % p
    
    #print(g_l_k_ls)
    #print(b_bc_rs)
    print("---------------------------------------------------------------")
    
        
    
